In [40]:
from pathlib import Path
from torch.utils.data import WeightedRandomSampler, random_split, DataLoader

from lifi.data.Loader import ImageDataset, LabelEnum
from lifi.train import get_label_probabilities

data_folder = Path("./images")
db = ImageDataset(data_folder)
train_db, test_db = random_split(db, [0.7, 0.3])
label_probas = get_label_probabilities(train_db)
sampler = WeightedRandomSampler(label_probas, len(train_db))
train_loader = DataLoader(train_db, 32, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_db, 32, shuffle=True, num_workers=4)

In [41]:
import torch
from torchvision.transforms import RandomApply

from lifi.data.Augmentinator import Augmentinator, default_transforms

x, y = train_db[0]
transform_dict = default_transforms(x.shape)
transforms = RandomApply(torch.nn.ModuleList([
    t for t in transform_dict.values()
]), p=0.1)


In [84]:
import torch.nn as nn
from torch.optim import Adam
from tqdm.notebook import tqdm

from lifi.models import BasicClassifier

net = BasicClassifier(num_classes=len(LabelEnum))

ce_loss = nn.CrossEntropyLoss()
net = net.cuda()
optim = Adam(net.parameters())

for _ in range(2):
    for x, y in tqdm(train_loader):
        optim.zero_grad()
        x = x.cuda()
        x = net.preprocess(x)
        x = transforms(x)
        y = y.cuda()
        y_hat = net(x)
        loss = ce_loss(y_hat, y)
        loss.backward()
        optim.step()
        print(loss)




  0%|          | 0/159 [00:00<?, ?it/s]

/home/joep/Code/Leafliction/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(2.0842, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8724, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5407, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.8125, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.9681, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6770, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4853, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7860, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4321, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4712, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2719, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3027, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1144, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2152, device='cuda:0', grad_fn=

  0%|          | 0/159 [00:00<?, ?it/s]

tensor(0.1026, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4684, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0298, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1305, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.2326, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1031, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.3223, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0581, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0803, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0041, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0338, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1373, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0165, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1196, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1431, device='cuda:0', grad_fn=

In [71]:
preds, gts = [], []

for x, y in tqdm(test_loader):
    x = x.cuda()
    x = net.preprocess(x)
    y = y.cuda()
    y_hat = torch.argmax(net(x), dim=1)
    preds.append(y_hat.cpu())
    gts.append(y.cpu())

  0%|          | 0/68 [00:00<?, ?it/s]

/home/joep/Code/Leafliction/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [72]:
import numpy as np

preds_np = np.array(torch.concat(preds).view(-1))
gts_np   = np.array(torch.concat(gts).view(-1))

In [76]:
(preds_np == gts_np).astype(float)

0.9834254143646409

In [85]:
from sklearn.metrics import classification_report
import pandas as pd

report = classification_report(gts_np, preds_np, target_names=[LabelEnum(i).name for i in range(len(LabelEnum))], output_dict=True)
pd.DataFrame.from_dict(report)

           Apple_Black_rot  Apple_healthy  ...    macro avg  weighted avg
precision         0.958333        0.98167  ...     0.980459      0.983568
recall            0.978723        0.98167  ...     0.981565      0.983425
f1-score          0.968421        0.98167  ...     0.980960      0.983452
support         188.000000      491.00000  ...  2172.000000   2172.000000

[4 rows x 11 columns]


In [24]:
import torch
from lifi.models import BasicClassifier
from lifi.data.labels import LabelEnum

net = BasicClassifier()
state_dict = torch.load('leaf_classifier.pt', map_location='cpu')
net.load_state_dict(state_dict)

<All keys matched successfully>

In [25]:
from PIL import Image
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import ToTensor

totens = ToTensor()
paf = "./correction/fix/Apple_Black_rot/Apple_Black_rot1.jpg"
im = totens(Image.open(paf))

pred = torch.softmax(net(net.preprocess(im.unsqueeze(0))), dim=1)[0]
{LabelEnum(i).name: p.item() for i, p in enumerate(pred)}

device = 'cpu'

x = im.to(device).unsqueeze(0)
x = net.preprocess(x)
y_hat = torch.argmax(torch.softmax(net(x), dim=1), dim=1)
print(LabelEnum(y_hat.item()).name)

Grape_Esca


In [26]:
from pathlib import Path
from torch.utils.data import DataLoader
from lifi.data.Loader import ImageDataset
from lifi.train import eval_model

data_folder = Path("./correction/fix/")
test_db = ImageDataset(data_folder)
test_loader = DataLoader(test_db, batch_size=1, shuffle=False)

Initialised ImageDataset on folder correction/fix with 10 images.


In [30]:
eval_model(test_loader, net, device='cpu')

Evaluating model:   0%|          | 0/10 [00:00<?, ?it/s]/home/joep/Code/Leafliction/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Evaluating model: 100%|██████████| 10/10 [00:04<00:00,  2.15it/s]


,Apple_Black_rot,Apple_healthy,Apple_rust,Apple_scab,Grape_Black_rot,Grape_Esca,Grape_healthy,Grape_spot,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
support,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,10.0,10.0


In [27]:
import pandas as pd
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import numpy as np

device = 'cpu'
preds, gts = [], []

net = net.to(device)
net = net.eval()

for x, y in tqdm(test_loader, desc="Evaluating model"):
    x = x.to(device)
    x = net.preprocess(x)
    y = y.to(device)
    y_hat = torch.argmax(torch.softmax(net(x), dim=1), dim=1)
    preds.append(y_hat.cpu())
    gts.append(y.cpu())
    print("pred: ", LabelEnum(y_hat.item()).name, LabelEnum(y.item()).name)


preds_np = np.array(torch.concat(preds).view(-1))
gts_np   = np.array(torch.concat(gts).view(-1))

report = classification_report(gts_np, preds_np, target_names=[LabelEnum(i).name for i in range(len(LabelEnum))], output_dict=True)
pd.DataFrame.from_dict(report)

Evaluating model:   0%|          | 0/10 [00:00<?, ?it/s]

pred:  Apple_scab Apple_scab
pred:  Apple_rust Apple_rust
pred:  Apple_healthy Apple_healthy
pred:  Apple_healthy Apple_healthy
pred:  Grape_spot Grape_spot
pred:  Grape_Black_rot Grape_Black_rot
pred:  Grape_Black_rot Grape_Black_rot
pred:  Apple_Black_rot Apple_Black_rot
pred:  Grape_healthy Grape_healthy
pred:  Grape_Esca Grape_Esca


,Apple_Black_rot,Apple_healthy,Apple_rust,Apple_scab,Grape_Black_rot,Grape_Esca,Grape_healthy,Grape_spot,accuracy,macro avg,weighted avg
precision,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
recall,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
f1-score,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
support,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,10.0,10.0


In [22]:
test_db.image_files[7]
im_l, y_l = test_db[7]

totens = ToTensor()
paf = "./correction/fix/Apple_Black_rot/Apple_Black_rot1.jpg"
im = totens(Image.open(paf))

In [23]:
# x = im.unsqueeze(0)
x = im_l.unsqueeze(0)
x = x.to(device)
x = net.preprocess(x)
y_hat = torch.argmax(torch.softmax(net(x), dim=1), dim=1)
preds.append(y_hat.cpu())
print(LabelEnum(y_hat.item()).name)
print(net(x))

Apple_Black_rot
tensor([[ 4.3443,  0.5088,  0.2857,  0.3990, -2.0597,  1.4670, -1.6564, -1.1900]],
       grad_fn=<AddmmBackward0>)


/home/joep/Code/Leafliction/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(1.)